In [1]:
!pip install sentence-transformers
!pip install faiss-cpu
!pip install llama-cpp-python[server]  # Install LLaMA loader (GGUF)
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
from google.colab import files

uploaded = files.upload()
pdf_path = next(iter(uploaded))
print("Uploaded file:", pdf_path)

Saving Medical_book.pdf to Medical_book.pdf
Uploaded file: Medical_book.pdf


In [4]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 54.4 MB/s eta 0:00:00


In [5]:
import pdfplumber

def extract_text_from_pdf(pdf_path, max_pages=50):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            if i >= max_pages:
                break
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

all_text = extract_text_from_pdf(pdf_path, max_pages=50)
print("Characters extracted:", len(all_text))

Characters extracted: 171999


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n", ".", " "],
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_text(all_text)
print("Number of chunks:", len(chunks))
print("Example chunk:\n", chunks[0][:200])

Number of chunks: 214
Example chunk:
 GALE
The
ENCYCLOPEDIA
M
EDICINE
of
SECOND EDITION
G ALE
The
ENCYCLOPEDIA
M
EDICINE
of
SECOND EDITION
V O L U M E
1
A-B
JACQUELINE L. LONGE, EDITOR
DEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR
The GALE Sin


In [8]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [9]:
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

# Use a small local model for embeddings
embedding_model = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Build FAISS index
vector_store = FAISS.from_texts(chunks, embedding_model)
print("FAISS index created using", len(chunks), "chunks.")

/tmp/ipython-input-2028976978.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS index created using 214 chunks.


In [10]:
# Create models folder and download GGUF model
!mkdir -p /content/models

!wget -O /content/models/mistral-7b-instruct.Q4_K_M.gguf \
   https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf

--2025-08-16 20:00:20--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.4, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/46/12/46124cd8d4788fd8e0879883abfc473f247664b987955cc98a08658f7df6b826/14466f9d658bf4a79f96c3f3f22759707c291cac4e62fea625e80c7d32169991?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.1.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.1.Q4_K_M.gguf%22%3B&Expires=1755378020&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NTM3ODAyMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy80Ni8xMi80NjEyNGNkOGQ0Nzg4ZmQ4ZTA4Nzk4ODNhYmZjNDczZjI0NzY2NGI5ODc5NTVjYzk4YTA4NjU4ZjdkZjZiODI2LzE0NDY2ZjlkNjU4YmY0YTc5Zjk2YzNmM2Yy

In [11]:
from llama_cpp import Llama

model_path = "/content/models/mistral-7b-instruct.Q4_K_M.gguf"

# Load model
llm = Llama(
    model_path=model_path,
    n_ctx=2048  # context window
)

print("Model loaded!")

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/models/mistral-7b-instruct.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                

Model loaded!


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | BMI2 = 1 | LLAMAFILE = 1 | OPENMP = 1 | REPACK = 1 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.rope.freq_base': '10000.000000', 'llama.context_length': '32768', 'general.name': 'mistralai_mistral-7b-instruct-v0.1', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_token_id': '1', 'llama.attention.head_count': '32', 'llama.block_count': '32', 'llama.attention.head_count_kv': '8', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama', 'general.file_type': '15'}
Using fallback chat format: llama-2


In [12]:
out = llm("Hello, who are you?", max_tokens=50)
print(out)

llama_perf_context_print:        load time =    3627.46 ms
llama_perf_context_print: prompt eval time =    3627.14 ms /     7 tokens (  518.16 ms per token,     1.93 tokens per second)
llama_perf_context_print:        eval time =   17907.90 ms /    24 runs   (  746.16 ms per token,     1.34 tokens per second)
llama_perf_context_print:       total time =   21548.68 ms /    31 tokens
llama_perf_context_print:    graphs reused =         23


{'id': 'cmpl-45fe6a64-77b0-4133-8f29-fdaa2f178056', 'object': 'text_completion', 'created': 1755374518, 'model': '/content/models/mistral-7b-instruct.Q4_K_M.gguf', 'choices': [{'text': ' What is your name?\nA: I am Mistral, a language model trained by the Mistral AI team.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 7, 'completion_tokens': 24, 'total_tokens': 31}}


In [13]:
def retrieve_chunks(question, top_k=3):
    # This gets top_k similar chunks
    docs = vector_store.similarity_search(question, k=top_k)
    return [doc for doc in docs]

In [14]:
def build_prompt(question, context_docs):
    context_text = "\n\n".join(doc.page_content for doc in context_docs)
    prompt = f"""You are a medical assistant.
Use the following medical context to answer the question.
Context:
{context_text}

Question: {question}

Answer in simple terms:
"""
    return prompt

In [16]:
def answer_from_pdf(question):
    docs = retrieve_chunks(question, top_k=2)
    final_prompt = build_prompt(question, docs)
    response = llm(final_prompt, max_tokens=200)
    answer_text = response["choices"][0]["text"].strip()
    return answer_text

In [17]:
print(answer_from_pdf("What is abdominal ultrasound?"))

Llama.generate: 1 prefix-match hit, remaining 515 prompt tokens to eval
llama_perf_context_print:        load time =    3627.46 ms
llama_perf_context_print: prompt eval time =  176594.08 ms /   515 tokens (  342.90 ms per token,     2.92 tokens per second)
llama_perf_context_print:        eval time =  116123.59 ms /   143 runs   (  812.05 ms per token,     1.23 tokens per second)
llama_perf_context_print:       total time =  292823.65 ms /   658 tokens
llama_perf_context_print:    graphs reused =        138


Abdominal ultrasound is a medical imaging technique that uses ultrasound waves to see inside your body. It is commonly used to check the size and shape of your organs, such as your liver and kidneys. It can also be used to see any problems in your digestive system, such as a blockage in your intestines. The procedure is usually done by a doctor with the assistance of a technologist who moves the ultrasound probe around your abdomen to get the best images. There are different types of ultrasound modes that can be used to obtain different views of the target areas. The procedure is generally safe and discomfort during the procedure is minimal.


In [ ]:
def answer_from_pdf(question):#for fast execution
    # Get relevant chunks
    docs = retrieve_chunks(question, top_k=1)  # only top 1 chunk to keep prompt small

    # Shorten the context for prompt to avoid slowness
    context_text = docs[0].page_content[:1000]

    prompt = f"""You are a medical assistant. Use the following context to answer the question briefly.

Context:
{context_text}

Question: {question}

Answer in 3-4 sentences:
"""
    # Generate shorter answer
    response = llm(prompt, max_tokens=60)
    answer_text = response["choices"][0]["text"].strip()
    return answer_text

In [18]:
chat_history = []

def chat():
    global chat_history
    print("Medical Chatbot Ready (type 'quit' to exit)")
    while True:
        user_q = input("You: ")
        if user_q.lower() in ["quit", "exit", "stop"]:
            print("Chatbot: Goodbye!")
            break

        # use your function
        answer = answer_from_pdf(user_q)

        # store
        chat_history.append((user_q, answer))

        print("Chatbot:", answer)

# run the chat
chat()

Medical Chatbot Ready (type 'quit' to exit)
You: Abdominal ultrasound


Llama.generate: 268 prefix-match hit, remaining 245 prompt tokens to eval
llama_perf_context_print:        load time =    3627.46 ms
llama_perf_context_print: prompt eval time =   83480.18 ms /   245 tokens (  340.74 ms per token,     2.93 tokens per second)
llama_perf_context_print:        eval time =  150859.06 ms /   199 runs   (  758.09 ms per token,     1.32 tokens per second)
llama_perf_context_print:       total time =  234491.90 ms /   444 tokens
llama_perf_context_print:    graphs reused =        192


Chatbot: Abdominal ultrasound is a medical imaging test that uses sound waves to create images of organs and tissues in the abdomen. There are four different modes of ultrasound used in medical imaging: A-mode, B-mode, M-mode, and Doppler ultrasound. The A-mode is the simplest type of ultrasound and scans a line through the body. The B-mode creates two-dimensional images of the organs and is used to measure distances within the body. The M-mode creates detailed images of the movement of the heart and other organs. Doppler ultrasound measures the speed of blood flow and can be used to diagnose conditions like heart disease.

Abdominal ultrasound is generally safe and painless, but there may be some discomfort during the procedure. It can be used to diagnose a wide range of conditions, including gallstones, cancer, spleen
You: exit
Chatbot: Goodbye!


In [19]:
vector_store.save_local("medical_faiss_index")